In [ ]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
import torch
import matplotlib.pyplot as plt
import nn_utils as nn_ut
import os
import utils as ut

In [ ]:
# setting latex style for plots
plt.rcParams['text.usetex'] = True

# fixing the seed
torch.manual_seed(29)

In [ ]:
# setting the payoff function f

K1 = 0.9
K2 = 1.2
def f(x):
    return torch.maximum(x - K1, torch.tensor(0.)) - torch.maximum(x - K2, torch.tensor(0.))

plt.plot(torch.arange(0.75,1.25,0.01), f(torch.arange(0.75,1.25,0.01)), label='payoff function')
plt.legend()
plt.show()

In [ ]:
# defining the reference measure as the marginal of a Black Scholes model and plotting a histogram at initial time
sigma = 0.20
t = .5
mu = torch.distributions.LogNormal(- 0.5 * sigma * sigma * t, sigma * math.sqrt(t))

samples = mu.sample([100000])
plt.hist(samples.detach().numpy(), 300, label='reference measure')
plt.legend()
plt.show()

In [ ]:
# analytic black scholes call price
def black_scholes_call(x, K, T, vol, rate):
    d1 = (np.log(x/K) + (rate + 0.5 * np.power(vol, 2)) * T) / (vol * np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return x * norm.cdf(d1) - K * np.exp(- rate * T) * norm.cdf(d2)

In [ ]:
# setting the maturities and the penalty power
maturities = np.linspace(0., 3./12, 21)
p = 3

# initializing the output dataframe
df_price = pd.DataFrame(index=t + maturities, columns=None)
df_price['Mat'] = t + maturities

# computing the black scholes price at each maturity
bs_prices = []
for i in range(maturities.shape[0]):
    call_long_tmp = black_scholes_call(x=1., K=K1, T=t + maturities[i], vol=sigma, rate=0.)
    call_short_tmp = black_scholes_call(x=1., K=K2, T=t + maturities[i], vol=sigma, rate=0.)
    bull_spread_tmp = call_long_tmp - call_short_tmp
    bs_prices.append(bull_spread_tmp)
df_price['BS price'] = bs_prices

In [ ]:
# cycling over maturities for the upper bound

upper_prices = [bs_prices[0]]
mc_err_up = [0]

for i in range(1,maturities.shape[0]):
    h = maturities[i]

    # defining the cost functional
    def cost(u):
        return h * torch.pow(torch.absolute(u) / math.sqrt(h), p)


    # defining the risk measure object
    width = 20
    depth = 4
    sample_size = 500
    risk_measure = nn_ut.MartRiskMeasure1d(f, cost, mu, torch.nn.ReLU, width, depth)

    # otpimizer
    optim = torch.optim.Adam(risk_measure.parameters(), lr=0.001)

    # training cycle
    epochs = 10000
    for i in range(epochs):
        optim.zero_grad()
        y = mu.sample([sample_size,1])
        risk = risk_measure(y)
        risk.backward()
        optim.step()

    risk_measure.eval()

    final_samples = mu.sample([100000,1])
    mc_rm = -risk_measure(final_samples)
    upper_prices.append(mc_rm.detach().numpy())
    mc_err = risk_measure.mc_err
    mc_err_up.append(mc_err.detach().numpy())

# appending the results to the dataframe
df_price['upper bound'] = upper_prices
df_price['mc err up'] = mc_err_up

In [ ]:
# cycling over maturities for the lower bound

lower_prices = [bs_prices[0]]
mc_err_down = [0]

for i in range(1,maturities.shape[0]):
    h = maturities[i]

    # defining the cost functional
    def cost(u):
        return h * torch.pow(torch.absolute(u) / math.sqrt(h), p)

    # defining the auxiliary payoff function
    def aux_f(x):
        return - f(x)

    # defining the risk measure object
    width = 20
    depth = 4
    sample_size = 500
    risk_measure = nn_ut.MartRiskMeasure1d(aux_f, cost, mu, torch.nn.ReLU, width, depth)

    # otpimizer
    optim = torch.optim.Adam(risk_measure.parameters(), lr=0.001)

    # training cycle
    epochs = 10000
    for i in range(epochs):
        optim.zero_grad()
        y = mu.sample([sample_size,1])
        risk = risk_measure(y)
        risk.backward()
        optim.step()

    risk_measure.eval()

    final_samples = mu.sample([100000,1])
    mc_rm = risk_measure(final_samples)
    lower_prices.append(mc_rm.detach().numpy())
    mc_err = risk_measure.mc_err
    mc_err_down.append(mc_err.detach().numpy())

# appending the results to the dataframe
df_price['lower bound'] = lower_prices
df_price['mc err down'] = mc_err_down

In [ ]:
# plotting the results
plt.plot(df_price['Mat'], df_price['BS price'], color='green', label="Black-Scholes")
plt.plot(df_price['Mat'], df_price['upper bound'], linestyle='--', color='red',label="Upper Bound")
plt.plot(df_price['Mat'], df_price['lower bound'], linestyle='-.', color='blue', label="Lower Bound")
plt.xlabel('Maturity')
plt.ylabel('Fair Value')
plt.ylim([0.06,0.17])
plt.legend()
plt.show()

In [ ]:
# saving the results as excel file
folder_out = 'output'
ut.check_dir(folder_out)

excel_out = os.path.join('output','bull_spread_0.5-0.75-20_500b.xlsx')
df_price.to_excel(excel_out)